In [66]:
import os
import pathlib
import json
import io
import tensorflow as tf
from random import shuffle
from shutil import rmtree
from object_detection.utils import label_map_util

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from xml.dom import minidom

import pandas as pd

In [67]:
PROJECT_CONFIG_PATH = 'workspace/road_damage/training-config.json'

In [68]:
def load_config():
    with open(PROJECT_CONFIG_PATH, 'r') as f:
        return json.load(f)
        

In [69]:
PROJECT_CONFIG = load_config()

In [70]:
def create_name_index():
    categories_map = label_map_util.create_categories_from_labelmap((PROJECT_CONFIG['PATH_TO_LABELS']))
    name_index = {}
    for cat in categories_map:
        name_index[cat['name']] = cat
    return name_index

In [71]:
def xml_to_csv(annotations):
    xml_list = []
    for idx, xml_file in enumerate(annotations):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        size = root.find('size')
        
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        
        filename = root.find('path').text.replace('\\','/')+'/'+root.find('filename').text
        count = 0
        for member in root.findall('object'):
            count = count + 1
            bndbox = member.find('bndbox')
            value = (filename,
                    width,
                    height,
                    member.find('name').text,
                    int(bndbox.find('xmin').text),
                    int(bndbox.find('ymin').text),
                    int(bndbox.find('xmax').text),
                    int(bndbox.find('ymax').text)
                    )
            xml_list.append(value)
        
        print('Processed {}. {} rows'.format(xml_file, count))
        
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax']
    return pd.DataFrame(xml_list, columns=column_name)

In [72]:
def create_tf_record(group,category_map):
    with tf.compat.v1.gfile.GFile(group.filename, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(category_map[row['class']]['id'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [73]:
def load_annotation(fname):
    tree = ET.parse(fname)
    root = tree.getroot()
    return root

In [74]:
def get_image_from_annotation(annotation: Element):
    return os.path.join(annotation.find('path').text, annotation.find('filename').text)

In [75]:
def validated_list(original_list):
    new_list = []
    for annotation_path in original_list:
        try:
            annotation = load_annotation(annotation_path)
            img_path = get_image_from_annotation(annotation)
            img = Image.open(img_path)
            width, height = img.size
            num_objects = 0
            for member_object in annotation.findall('object'):
                num_objects += 1
                bnd_box = member_object.find('bndbox')
                xmin = int(bnd_box.find('xmin').text)
                ymin = int(bnd_box.find('ymin').text)
                xmax = int(bnd_box.find('xmax').text)
                ymax = int(bnd_box.find('ymax').text)
                if xmin>width or xmax>width or ymin>height or ymax>height:
                    raise Exception('Bounding box dimensions greater than max width or height')
            if num_objects > 0:
                new_list.append(annotation_path)
        except BaseException as e:
            print('Skipping {} because {}'.format(annotation_path, e))
    return new_list

In [76]:
def perform_partition(argv):    
    print(argv)
    split_percentage = 0.9
    annotation_names = [os.path.join(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'],f) for f in os.listdir(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'])]
    annotation_names = validated_list(annotation_names)
    shuffle(annotation_names)
    num_annotations = len(annotation_names)
    num_train = int(num_annotations*split_percentage)
    train_annotations, test_annotations = annotation_names[0:num_train],annotation_names[num_train:]

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'])

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'])

    def generate_record_and_csv(source_base: str, target_base: str, annotations):
        if not os.path.exists(target_base):
            os.makedirs(target_base)

        xml_df = xml_to_csv(annotations)
        xml_df.to_csv(os.path.join(target_base,'annotations_csv.csv'), index=None)
        print('Finished outputting {}'.format(os.path.join(target_base,'annotations_csv.csv')))
        
        # Finished outputting the CSV record above, now converting it to a record
        with tf.compat.v1.python_io.TFRecordWriter(os.path.join(target_base,'tensorflow_record.record')) as writer:
            
            data = namedtuple('data', ['filename', 'object'])
            gb = xml_df.groupby('filename')
            grouped = [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]
            
            category_map = create_name_index()
            for group in grouped:
                tf_record = create_tf_record(group, category_map)
                writer.write(tf_record.SerializeToString())


    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'], 
        train_annotations)

    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'], 
        test_annotations)
    
    print('Vini','Vidi','Vici')

In [77]:
tf.compat.v1.app.run(perform_partition)

['c:\\users\\letlhogonolo segoe\\pycharmprojects\\models\\my-goodness-experiments\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Letlhogonolo Segoe\\AppData\\Roaming\\jupyter\\runtime\\kernel-384620f0-1073-4711-9afe-3ab513d5c32d.json']
Processed D:/university/datasets/Potholes/consolidatedAnnotations\11236.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7141.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2697.xml. 7 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\1018.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\10192.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4484.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\5884.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2428.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8662.xml. 3 rows

Processed D:/university/datasets/Potholes/consolidatedAnnotations\8415.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8666.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\57.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8993.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\95.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\3433.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7082.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\397.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4130.xml. 5 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8847.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8921.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\662.xml. 2 rows
Processed 

Processed D:/university/datasets/Potholes/consolidatedAnnotations\723.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\11235.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\6069.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\1850.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\703.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\5770.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4524.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\530.xml. 17 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\9256.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\5274.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\11202.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7255.xml. 1 rows
Proc

Processed D:/university/datasets/Potholes/consolidatedAnnotations\7557.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\9557.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\1157.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\10631.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7797.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\9094.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\769.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\776.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4009.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\1228.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7303.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7497.xml. 1 rows
Proce

Processed D:/university/datasets/Potholes/consolidatedAnnotations\8511.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2696.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\5546.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2285.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\408.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7423.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2698.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2373.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\914.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8761.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4010.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\795.xml. 3 rows
Process

...

Processed D:/university/datasets/Potholes/consolidatedAnnotations\7359.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\3503.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4223.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8769.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\837.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\6353.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\2744.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\1453.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\6931.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\697.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\3043.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8672.xml. 2 rows
Proces

Processed D:/university/datasets/Potholes/consolidatedAnnotations\7095.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\5886.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\6280.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\642.xml. 2 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7228.xml. 7 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7758.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\807.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\4674.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\9036.xml. 4 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8035.xml. 3 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\7211.xml. 1 rows
Processed D:/university/datasets/Potholes/consolidatedAnnotations\8280.xml. 4 rows
Proces

I0607 18:40:50.444715 19280 label_map_util.py:120] Ignore item 0 since it falls outside of requested label range.


Vini Vidi Vici


SystemExit: 